In [2]:
%region "us-east-1"

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Previous region: None
Setting new region to: us-east-1
Region is set to: us-east-1


In [3]:
# %iam_role arn:aws:iam::111222333:role/my-interactive-session-role-name

In [5]:
# Set `glue_role_arn=arn:aws:iam::1112223333:role/my-interactive-session-role-name` in ~/.aws/credentials
%profile "de"

Previous profile: default
Setting new profile to: de


In [7]:
#%connections eblocks3csv
%connections data-warehouse

Connections to be included:
data-warehouse


In [9]:
%glue_version 3.0

Setting Glue version to: 3.0


In [11]:
%number_of_workers 2

Previous number of workers: 5
Setting new number of workers to: 2


In [13]:
%iam_role arn:aws:iam::1234567890123:role/LakeFormationWorkflowRole

Current iam_role is None
iam_role has been set to arn:aws:iam::1234567890123:role/LakeFormationWorkflowRole.


In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Authenticating with profile=de
glue_role_arn defined by user: arn:aws:iam::1234567890123:role/LakeFormationWorkflowRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: 53d36510-109b-4b50-86b0-78f414cd59e9
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
Waiting for session 53d36510-109b-4b50-86b0-78f414cd59e9 to get into ready status...
Session 53d36510-109b-4b50-86b0-78f414cd59e9 has been created




In [1]:
df_s3 = glueContext.create_dynamic_frame.from_catalog(
        database = "eblocks3csv", 
        table_name = "inventory_item" 
    )

In [2]:
df_s3.printSchema()

root
|-- id: string
|-- consigner_id: string
|-- listed_by_user_id: string
|-- location_id: string
|-- auction_location_id: string
|-- model_id: string
|-- sub_model_id: string
|-- trim_id: string
|-- primary_photo_id: string
|-- condition_report_id: string
|-- type: string
|-- status: string
|-- condition: string
|-- tag_line: string
|-- trim_name: string
|-- year: long
|-- stock_number: string
|-- description: string
|-- completion_percentage: long
|-- created: string
|-- updated: string
|-- verified: choice
|    |-- long
|    |-- string
|-- verified_by: string
|-- verified_extensive_vehicle_condition: choice
|    |-- long
|    |-- string
|-- archived: choice
|    |-- long
|    |-- string
|-- inspection_notes: string
|-- version: long
|-- pickup_location_id: string
|-- checked_in_by_user_id: string
|-- checked_out_by_user_id: string
|-- checked_in_datetime: string
|-- checked_out_datetime: string
|-- chrome_tpms: choice
|    |-- long
|    |-- string
|-- external_id: string
|-- item_s

In [4]:

df_s3.toDF().createOrReplaceTempView("inventory_item")

In [5]:
spark.sql("select * from inventory_item").show(2)

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+----------+---------+--------+-------------------+----+------------+--------------------+---------------------+-------------------+-------------------+---------+-----------+------------------------------------+---------+----------------+-------+--------------------+---------------------+----------------------+-------------------+--------------------+-----------+-----------+-----------+------------------+---------+---------------+--------------+--------------------+--------------------+-----------------+----------------+--------------------+---------------------------------+--------------+------------+-----------------+-------------+-----------+-----+----------+--------------+--------------+----------+------------+-----------------+-------------------+---------+-----------

In [6]:
#https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-dynamic-frame-reader.html

df_redshift = glueContext.create_dynamic_frame.from_catalog(
        database = "data_warehouse", 
        table_name = "data_warehouse_tableau_control_kpi", 
        redshift_tmp_dir = "s3://einc-glue-redshift-tmp-dev/temp/")

In [7]:
df_redshift.printSchema()

root
|-- id: int
|-- kpi_name: string
|-- actor: string
|-- object_id: string
|-- aggregation_date_field: string
|-- first_time_ind: boolean
|-- where_clause: string


In [10]:
# https://docs.aws.amazon.com/glue/latest/dg/aws-glue-programming-etl-glue-data-catalog-hive.html

df_redshift.toDF().createOrReplaceTempView("data_warehouse_tableau_mv_aug_auction_item")

In [11]:
spark.sql("select * from data_warehouse_tableau_mv_aug_auction_item").show(2)

+---+-------------+------+--------------------+----------------------+--------------+--------------------+
| id|     kpi_name| actor|           object_id|aggregation_date_field|first_time_ind|        where_clause|
+---+-------------+------+--------------------+----------------------+--------------+--------------------+
| 12|Unique Listed|seller|  inventory_item_vin|  auction_item_ran_...|         false|                null|
|  1|         Sold|seller|auction_item_invo...|  auction_item_invo...|         false| and auction_item...|
+---+-------------+------+--------------------+----------------------+--------------+--------------------+
only showing top 2 rows
